In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Video Analysis with Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/video_analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fvideo-analysis%2Fvideo_analysis.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>       
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/video_analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/video-analysis/video_analysis.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner) |

## Getting Started


### Install Vertex AI SDK for Python

In [1]:
%pip install --upgrade --user -q google-cloud-aiplatform

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
# Define project information
PROJECT_ID = "qwiklabs-gcp-02-b546d8001349"  # @param {type:"string"}
LOCATION = "us-east4"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries


In [3]:
from IPython.display import Markdown, Video, display
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

### Load the Gemini 1.5 Pro model

Gemini 1.5 Pro (`gemini-1.5-pro`) is a multimodal model that supports multimodal prompts. You can include text, image(s), PDFs, audio, and video in your prompt requests and get text or code responses.

In [4]:
model = GenerativeModel("gemini-1.5-pro")

generation_config = GenerationConfig(temperature=1, top_p=0.95, max_output_tokens=8192)

safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

### Helper functions

In [5]:
def get_url_from_gcs(gcs_uri: str) -> str:
    # converts GCS uri to url for display.
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )

## Sentiment Analysis from Video and Audio

For this example, we will be analyzing a video of the first televised US presidential debate between John F. Kennedy and Richard Nixon. This debate is largely believed to have been critical in securing JFK's victory due to his calm and personable demeanor for the broadcast.

The video is an hour long, which is roughly equivalent to 1 Million Tokens for Gemini 1.5 Pro.

In [6]:
video_analysis_prompt = """You are an expert in politics and history. Provide a detailed analysis of the video including each speakers facial expressions and mood at key points."""

In [7]:
# Load file directly from Google Cloud Storage
video_uri = "gs://github-repo/video/KennedyNixon1960PresidentialDebate.mp4"

# Load contents
contents = [
    Part.from_uri(
        uri=video_uri,
        mime_type="video/mp4",
    ),
    video_analysis_prompt,
]

# Display the Video
display(Video(get_url_from_gcs(video_uri)))

Note: due to the length of the video, this will take a few minutes to complete. If you receive a 500 error, please wait a few moments and run the cell again.

In [8]:
# Send to Gemini
response = model.generate_content(contents, generation_config=generation_config)

# Display results
display(Markdown(response.text))

The video features a black and white recording of the first-ever televised presidential debate in 1960 between Democratic candidate John F. Kennedy and Republican candidate Richard Nixon. The debate is moderated by Howard K. Smith and the panel of correspondents includes Sander Vanocur (NBC News), Charles Warren (Mutual News), Stuart Novins (CBS News), and Bob Fleming (ABC News).

**Kennedy's opening statement:**

* **Mood:** Kennedy appears confident, passionate, and determined to convey a sense of urgency to the American people. His tone is assertive, and he frequently makes eye contact with the camera. 
* **Facial Expressions:** He maintains a composed and serious expression throughout his statement, occasionally blinking slowly for emphasis. His gestures are deliberate and controlled, emphasizing key points in his speech.
* **Key points:**
    * He draws a parallel between the 1860 election (Lincoln) and the 1960 election, stating that the world's survival is at stake.
    * He emphasizes the need for America to do more in terms of economic growth, national strength, and education.
    * He criticizes the lack of progress made during the previous administration and calls for effective governmental action to address national challenges.
    * He underscores the importance of upholding constitutional rights for all Americans, particularly African Americans.
    * He stresses the urgency of the situation, using phrases like "I'm not satisfied" and "I don't think we're doing enough."

**Nixon's opening statement:**

* **Mood:** Nixon also appears determined but projects a more measured and calm demeanor compared to Kennedy's fiery delivery. He seems to focus more on refuting Kennedy's claims rather than presenting a strong, independent vision. 
* **Facial Expressions:** He maintains a neutral expression for the most part, but a slight frown becomes evident when he directly addresses Kennedy's statements. His eyes occasionally dart between the camera and the panel of correspondents.
* **Key points:**
    * He acknowledges the importance of the domestic issues highlighted by Kennedy but emphasizes their connection to international affairs and the need to stay ahead in the global competition.
    * He challenges Kennedy's claims about America standing still by providing statistics indicating progress in areas like education, hydroelectric power, and hospital construction.
    * He argues that his policies are more effective than Kennedy's and stresses the importance of individual initiative.
    * He criticizes Kennedy's proposals for increasing federal spending and argues for fiscally responsible solutions.
    * He subtly attempts to portray Kennedy as inexperienced and suggests that his own experience in the executive branch makes him a better candidate.

**Q&A session:**

* **Kennedy:** Kennedy continues to display confidence and composure while answering questions. He provides specific examples to support his arguments and confidently refutes Nixon's claims when necessary. He maintains a consistent tone and a focused expression, showing a grasp of the issues at hand.
* **Nixon:** Nixon appears more defensive during the Q&A session, particularly when responding to questions about his record and experience. He seems to be more focused on countering Kennedy's points than articulating his own vision or plans.  He appears somewhat flustered when challenged on specific details about his policies. 

**Overall Analysis:**

The video highlights the stark difference in style and approach between the two candidates. Kennedy's charismatic delivery and direct gaze at the camera likely resonated with television viewers, while Nixon's more restrained and defensive demeanor may have appeared less appealing on screen. This first televised debate is widely believed to have significantly influenced the outcome of the election, with Kennedy's performance contributing to his narrow victory. The debate marked a turning point in American politics, showcasing the power of television and image in shaping public opinion. 
